In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from numpy.typing import NDArray
import os


Preprocessing

In [8]:

data_dir = 'data'
data = pd.read_csv(os.path.join(data_dir, 'UNSW_NB15_training-set.csv'))
columns = data.columns

In [9]:
# Filter for specific filenames
acceptable = ['1.csv', '2.csv', '3.csv', '4.csv']

for filename in os.listdir(data_dir):
    if any(filename.endswith(suffix) for suffix in acceptable):
        file_path = os.path.join(data_dir, filename)
        df = pd.read_csv(file_path, header=0, names=columns, skiprows=1, low_memory=False)
        data = pd.concat([data, df], ignore_index=True)




In [10]:
data.shape

(2715380, 45)

In [14]:
cleaned_data = data.drop(['attack_cat', 'service', 'id' ], axis=1)        
clean_data = cleaned_data.dropna(axis=0, thresh=int(data.columns.__len__() * 0.80))



In [24]:
cleaned_data

,dur,proto,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,0.121478,tcp,FIN,6,4,258,172,74.08749,252.000000,254.000000,...,1.0,1,1,0,0,0,1,1,0,0
1,0.649902,tcp,FIN,14,38,734,42014,78.473372,62.000000,252.000000,...,1.0,1,2,0,0,0,1,6,0,0
2,1.623129,tcp,FIN,8,16,364,13186,14.170161,62.000000,252.000000,...,1.0,1,3,0,0,0,2,6,0,0
3,1.681642,tcp,FIN,12,12,628,770,13.677108,62.000000,252.000000,...,1.0,1,3,1,1,0,2,1,0,0
4,0.449454,tcp,FIN,10,6,534,268,33.373826,254.000000,252.000000,...,2.0,1,40,0,0,0,2,39,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715375,FIN,0.087306,1828,31,29,1,2,ftp-data,24465.671880,146610.765600,...,NaN,,1,2,3,3,1,1,3,0
2715376,CON,0.365058,346,31,29,2,2,ftp,8743.816406,6333.240234,...,2.0,2,2,2,2,2,2,2,2,0
2715377,CON,6.335154,2088,31,29,7,9,ftp,2204.839844,2549.582764,...,2.0,2,2,2,4,2,2,2,2,0
2715378,CON,2.200934,166054,31,29,2,57,http,12496.513670,598375.062500,...,NaN,,1,1,2,4,2,2,2,0


In [129]:
from sklearn.impute import KNNImputer

In [3]:
data.memory_usage()

NameError: name 'data' is not defined

In [131]:

categorical_columns = ['proto', 'service', 'state'] # Categorical columns to be converted to numerical\
data = pd.get_dummies(data, columns=categorical_columns) # Convert categorical columns to numerical
labels = data['label'].values # Extract labels
features = data.drop(columns=['label', 'attack_cat'], axis=1) # Extract features


''' Training and Testing Data '''
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels) # Split data into training and testing sets
scaler = StandardScaler() # Initialize scaler
X_train = scaler.fit_transform(X_train) # Fit and transform training data
X_test = scaler.transform(X_test) # Transform testing data



In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 6. Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(Y_test, dtype=torch.float32)

# 7. Create TensorDataset for convenient data handling
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

# 8. Use DataLoader to batch and shuffle the data
batch_size = 64  # industry-standard batch sizes are powers of 2 (32, 64, 128, etc.)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F



In [ ]:

# 9. Define the neural network architecture
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        # Define layers:
        self.fc1 = nn.Linear(input_size, 64)   # fully connected layer 1: input -> 64 hidden units
        self.fc2 = nn.Linear(64, 1)           # fully connected layer 2: 64 -> 1 output
    
    def forward(self, x):
        # Define the forward pass (how data moves through the network)
        x = F.relu(self.fc1(x))   # apply ReLU activation after first layer
        x = self.fc2(x)           # second layer (output logits)
        return x

# 10. Initialize the model
input_dim = X_train_tensor.shape[1]   # number of features after encoding
model = Net(input_dim)
print(model)


Net(
  (fc1): Linear(in_features=195, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [ ]:
import torch.optim as optim

In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = torch.device('cpu')
model.to(device)


Net(
  (fc1): Linear(in_features=195, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr= 0.001)


In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    
    for batch_X, batch_Y in train_loader:
        batch_X = batch_X.to(device)
        batch_Y = batch_Y.to(device)
        
        outputs = model(batch_X)
        outputs = outputs.view(-1)
        
        loss = criterion(outputs, batch_Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * batch_X.size(0)
        epoch_loss = running_loss / len(train_dataset)
        
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        

Epoch [1/100], Loss: 0.0524
Epoch [2/100], Loss: 0.0520
Epoch [3/100], Loss: 0.0515
Epoch [4/100], Loss: 0.0509
Epoch [5/100], Loss: 0.0506
Epoch [6/100], Loss: 0.0504
Epoch [7/100], Loss: 0.0496
Epoch [8/100], Loss: 0.0491
Epoch [9/100], Loss: 0.0492
Epoch [10/100], Loss: 0.0485
Epoch [11/100], Loss: 0.0481
Epoch [12/100], Loss: 0.0480
Epoch [13/100], Loss: 0.0479
Epoch [14/100], Loss: 0.0474
Epoch [15/100], Loss: 0.0474
Epoch [16/100], Loss: 0.0473
Epoch [17/100], Loss: 0.0465
Epoch [18/100], Loss: 0.0463
Epoch [19/100], Loss: 0.0460
Epoch [20/100], Loss: 0.0463
Epoch [21/100], Loss: 0.0461
Epoch [22/100], Loss: 0.0459
Epoch [23/100], Loss: 0.0452
Epoch [24/100], Loss: 0.0452
Epoch [25/100], Loss: 0.0447
Epoch [26/100], Loss: 0.0447
Epoch [27/100], Loss: 0.0447
Epoch [28/100], Loss: 0.0446
Epoch [29/100], Loss: 0.0437
Epoch [30/100], Loss: 0.0441
Epoch [31/100], Loss: 0.0436
Epoch [32/100], Loss: 0.0442
Epoch [33/100], Loss: 0.0436
Epoch [34/100], Loss: 0.0432
Epoch [35/100], Loss: 0

In [ ]:
model.eval()



Net(
  (fc1): Linear(in_features=195, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
correct = 0
total = 0
TP = FP = FN = 0

with torch.no_grad():
    for batch_X, batch_Y in test_loader:
        batch_X = batch_X.to(device)
        batch_Y = batch_Y.to(device)
        
        outputs = model(batch_X)
        
        probs = torch.sigmoid(outputs)
        preds = (probs >= 0.5).float()
                # Flatten predictions and labels to 1D
        preds = preds.view(-1)
        labels = batch_Y.view(-1)
        # Count correct predictions
        correct += (preds == labels).sum().item()
        total   += labels.size(0)
        
        TP += ((preds == 1) & (labels == 1)).sum().item()
        FP += ((preds == 1) & (labels == 0)).sum().item()
        FN += ((preds == 0) & (labels == 1)).sum().item()

accuracy = correct / total
precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

print(f"Test Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1_score*100:.2f}%")


Test Accuracy: 98.15%


In [ ]:
torch.save(model.state_dict(), "model.pth")